In [10]:
from pathlib import Path
import torch
torch.set_printoptions(precision=10)

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from models.ranked_transformer import HsqcRankedTransformer

from models.optional_input_ranked_transformer import OptionalInputRankedTransformer
from datasets.optional_2d_folder_dataset import OptionalInputDataModule
import yaml
from pytorch_lightning.loggers import TensorBoardLogger
torch.set_float32_matmul_precision('medium')

In [11]:
from datasets.dataset_utils import specific_radius_mfp_loader


Here we compare models' performance over 7 kinds of inputs


In [12]:
# load model 

# model_path = Path("/root/MorganFP_prediction/reproduce_previous_works/weird_H_and_tautomer_cleaned/flexible_models_best_FP/r0_r2_FP_trial_2/")
model_path = Path("/root/MorganFP_prediction/reproduce_previous_works/weird_H_and_tautomer_cleaned/train_on_all_data_possible/only_hsqc_trial_1/")

# checkpoint_path = model_path / "checkpoints/epoch=42-all_inputs.ckpt"
checkpoint_path = model_path / "checkpoints/epoch=21-step=37708.ckpt"


hyperpaerameters_path = model_path / "hparams.yaml"




with open(hyperpaerameters_path, 'r') as file:
    hparams = yaml.safe_load(file)
    
FP_building_type = hparams['FP_building_type'].split("_")[-1]
only_2d = not hparams['use_oneD_NMR_no_solvent']
specific_radius_mfp_loader.setup(only_2d=only_2d,FP_building_type=FP_building_type)
specific_radius_mfp_loader.set_max_radius(int(hparams['FP_choice'].split("_")[-1][1:]), only_2d=only_2d)
del hparams['checkpoint_path'] # prevent double defition of checkpoint_path



In [13]:
hparams['use_peak_values'] = False

hparams['use_Jaccard'] = True

In [14]:
model = HsqcRankedTransformer.load_from_checkpoint(checkpoint_path, **hparams)
model.change_ranker_for_testing()
# model.change_ranker_for_testing(test_ranking_set_path = "/workspace/ranking_sets_cleaned_by_inchi/SMILES_R0_to_R4_reduced_FP_ranking_sets_only_all_info_molecules/test/rankingset.pt")
model=model.to("cuda")

Using jaccard:  True


Initialized SignCoordinateEncoder[384] with dims [180, 180, 24] and 2 positional encoders. 24 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args


In [15]:
datamodule = OptionalInputDataModule(dir="/workspace/SMILES_dataset", FP_choice=hparams["FP_choice"], input_src=["HSQC", "oneD_NMR"], batch_size=hparams['bs'], parser_args=hparams)

datamodule.setup("test")
loader_all_inputs, loader_HSQC_H_NMR, loader_HSQC_C_NMR, loader_only_hsqc, loader_only_1d, loader_only_H_NMR, loader_only_C_NMR = \
    datamodule.test_dataloader()
    


In [16]:
trainer = Trainer()
metric_to_focus = [
    'rank_1',
    'rank_5',
    "mean_rank",
    "cos",
    'f1'    
]


def show_model_performance(model, out_file ="temp_model_show.txt"):
    loaders = [loader_all_inputs, loader_HSQC_H_NMR, loader_HSQC_C_NMR, loader_only_hsqc, loader_only_1d, loader_only_H_NMR, loader_only_C_NMR]
    names = ["all_inputs", "HSQC_H_NMR", "HSQC_C_NMR", "only_hsqc", "only_1d", "only_H_NMR", "only_C_NMR"]
    # names = ["only_hsqc"]
    with open(out_file, "w") as file:
        for loader, name in zip(loaders, names):
            result =  trainer.test(model, dataloaders=loader)
            file.write(f"\n{name:10}: ")
            for m in metric_to_focus:
                file.write(f"{m}: {result[0]['test/mean_'+m]:.4f}, ")


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [17]:
show_model_performance(model);

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 61/61 [00:13<00:00,  4.62it/s]{'on_epoch': True, 'sync_dist': False} 


Testing DataLoader 0: 100%|██████████| 61/61 [00:13<00:00,  4.58it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    test/mean_accuracy     │     0.984584391117096     │
│   test/mean_active_bits   │     90.49066162109375     │
│     test/mean_ce_loss     │    0.07743391394615173    │
│       test/mean_cos       │    0.6511443257331848     │
│       test/mean_f1        │    0.6233009099960327     │
│     test/mean_jaccard     │    0.4650381803512573     │
│    test/mean_mean_rank    │     60.73246383666992     │
│    test/mean_neg_loss     │   0.009024604223668575    │
│    test/mean_pos_loss     │    0.06840930879116058    │
│  test/mean_pos_neg_loss   │    0.07743391394615173    │
│    test/mean_precision    │    0.8672966361045837     │
│     test/mean_rank_1      │     0.389962762594223     │
│     test/mean_rank_10     │    0.6820147037506104     │
│     test/mean_rank_5      │    0.6014094352722168     │
│     test/mean_recall      │    0.4872291684150696     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 61/61 [00:12<00:00,  4.71it/s]{'on_epoch': True, 'sync_dist': False} 


Testing DataLoader 0: 100%|██████████| 61/61 [00:12<00:00,  4.71it/s]


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    test/mean_accuracy     │    0.9760757684707642     │
│   test/mean_active_bits   │    28.884084701538086     │
│     test/mean_ce_loss     │    0.18370100855827332    │
│       test/mean_cos       │    0.31798550486564636    │
│       test/mean_f1        │    0.22829847037792206    │
│     test/mean_jaccard     │    0.13196267187595367    │
│    test/mean_mean_rank    │      1658.064453125       │
│    test/mean_neg_loss     │   0.006232621148228645    │
│    test/mean_pos_loss     │    0.17746838927268982    │
│  test/mean_pos_neg_loss   │    0.18370100855827332    │
│    test/mean_precision    │    0.7508037686347961     │
│     test/mean_rank_1      │   0.0002561475266702473   │
│     test/mean_rank_10     │   0.004610655829310417    │
│     test/mean_rank_5      │   0.003329918021336198    │
│     test/mean_recall      │    0.13486836850643158    │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 61/61 [00:12<00:00,  4.70it/s]{'on_epoch': True, 'sync_dist': False} 


Testing DataLoader 0: 100%|██████████| 61/61 [00:12<00:00,  4.70it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    test/mean_accuracy     │    0.9752503037452698     │
│   test/mean_active_bits   │    14.112960815429688     │
│     test/mean_ce_loss     │    0.1984386444091797     │
│       test/mean_cos       │    0.24347227811813354    │
│       test/mean_f1        │    0.13422390818595886    │
│     test/mean_jaccard     │    0.0734238252043724     │
│    test/mean_mean_rank    │    1918.9146728515625     │
│    test/mean_neg_loss     │   0.003103332594037056    │
│    test/mean_pos_loss     │    0.19533531367778778    │
│  test/mean_pos_neg_loss   │    0.1984386444091797     │
│    test/mean_precision    │    0.8520639538764954     │
│     test/mean_rank_1      │   0.0005122950533404946   │
│     test/mean_rank_10     │   0.003329918021336198    │
│     test/mean_rank_5      │   0.0017930327448993921   │
│     test/mean_recall      │    0.07326986640691757    │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [ ]:
# spectra-flexible model
# usually not gonna use it because it tests right away after training 

# load model 

model_path = Path("/root/MorganFP_prediction/reproduce_previous_works/average_3/model_sizes/flexible_384_trail1")
hyperpaerameters_path = model_path / "hparams.yaml"

checkpoint_path = model_path / "checkpoints/epoch=41-step=35994.ckpt"
with open(hyperpaerameters_path, 'r') as file:
    hparams = yaml.safe_load(file)
del hparams['checkpoint_path'] # prevent double defition of checkpoint_path
hparams['use_peak_values'] = False

model = OptionalInputRankedTransformer.load_from_checkpoint(checkpoint_path, **hparams)
model.change_ranker_for_testing()

flexible_model_result = trainer.test(model, datamodule)


Using jaccard:  False


AssertionError: /workspace/ranking_sets_cleaned_by_inchi/SMILES_R0_to_R4_reduced_FP_ranking_sets_only_all_info_molecules/val/rankingset.pt does not exist

In [ ]:

names = ["all_inputs", "HSQC_H_NMR", "HSQC_C_NMR", "only_hsqc", "only_1d", "only_H_NMR", "only_C_NMR"]
out_file = "temp_model_show.txt"
with open(out_file, "w") as file:
    for name in  names:
        file.write(f"\n{name:10}: ")
        for m in metric_to_focus:
            file.write(f"{m}: {flexible_model_result[0]['test_mean_'+m+'_'+name]:.4f}, ")


In [ ]:
flexible_model_result[0] .keys()    


dict_keys(['test_mean_ce_loss_all_inputs', 'test_mean_pos_loss_all_inputs', 'test_mean_neg_loss_all_inputs', 'test_mean_pos_neg_loss_all_inputs', 'test_mean_cos_all_inputs', 'test_mean_active_bits_all_inputs', 'test_mean_f1_all_inputs', 'test_mean_precision_all_inputs', 'test_mean_recall_all_inputs', 'test_mean_accuracy_all_inputs', 'test_mean_mean_rank_all_inputs', 'test_mean_rank_1_all_inputs', 'test_mean_rank_5_all_inputs', 'test_mean_rank_10_all_inputs', 'test_mean_ce_loss_HSQC_H_NMR', 'test_mean_pos_loss_HSQC_H_NMR', 'test_mean_neg_loss_HSQC_H_NMR', 'test_mean_pos_neg_loss_HSQC_H_NMR', 'test_mean_cos_HSQC_H_NMR', 'test_mean_active_bits_HSQC_H_NMR', 'test_mean_f1_HSQC_H_NMR', 'test_mean_precision_HSQC_H_NMR', 'test_mean_recall_HSQC_H_NMR', 'test_mean_accuracy_HSQC_H_NMR', 'test_mean_mean_rank_HSQC_H_NMR', 'test_mean_rank_1_HSQC_H_NMR', 'test_mean_rank_5_HSQC_H_NMR', 'test_mean_rank_10_HSQC_H_NMR', 'test_mean_ce_loss_HSQC_C_NMR', 'test_mean_pos_loss_HSQC_C_NMR', 'test_mean_neg_loss_